In [8]:
import pathlib
from os import listdir

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
big_genres_df = pd.read_pickle("../../data/big_genres_tracks_df.pickle")

### Проверка наличия всех треков

In [5]:
all_tracks_ids = set(big_genres_df["track_id"])
tracks_found = []
dir_parts = pathlib.Path(
    "/home/kiri/Kiri_coding/SpotyParser/notebooks/data/big_genres_tracks_analysis/"
)

for part_dir in listdir(dir_parts):
    for track in listdir(dir_parts / part_dir):
        path_to_track = dir_parts / part_dir / track
        track_id = path_to_track.stem
        tracks_found.append(track_id)

tracks_found = set(tracks_found)
tracks_left = all_tracks_ids.difference(tracks_found)

#### Доброска не найденных треков (дальнейшная докачка через скрипты)

In [6]:
if tracks_left:
    df_left = big_genres_df[big_genres_df["track_id"].isin(tracks_left)]
    df_left = pd.DataFrame(df_left["track_id"])
    splitted = np.array_split(df_left, 8)
    for i, df_split in enumerate(splitted):
        df_split.to_csv(f"../scripts/data/big_genres_chunks/chunk_{i+1}.csv")
else:
    print("Все треки были скачаны")

Все треки были скачаны


### Объединение треков в датасет

In [12]:
dir_parts = pathlib.Path(
    "/home/kiri/Kiri_coding/SpotyParser/notebooks/data/big_genres_tracks_analysis/"
)

rows = []
for part_dir in tqdm(listdir(dir_parts)):
    for track in listdir(dir_parts / part_dir):
        to_row = []
        path_to_track = dir_parts / part_dir / track
        track_id = path_to_track.stem
        to_row.append(track_id)

        a_analysis = pd.read_pickle(path_to_track)
        if "track" not in a_analysis:
            continue
        trackdata = a_analysis["track"]
        keys_to_append = [
            "duration",  # in seconds
            "end_of_fade_in",
            "start_of_fade_out",
            "tempo_confidence",
            "time_signature",
            "time_signature_confidence",
            "key_confidence",
            "mode_confidence",
        ]
        for key in keys_to_append:
            to_row.append(trackdata[key])
        rows.append(to_row)

  0%|          | 0/16 [00:00<?, ?it/s]

In [21]:
analysis_df = pd.DataFrame(
    data=rows,
    columns=[
        "track_id",
        "duration_seconds",  # in seconds
        "end_of_fade_in",
        "start_of_fade_out",
        "tempo_confidence",
        "time_signature",
        "time_signature_confidence",
        "key_confidence",
        "mode_confidence",
    ],
)

analysis_df = analysis_df.drop_duplicates()

In [22]:
analysis_df.to_pickle("../../data/big_genres_audio_analysis_df.pickle")

### Объединение датасета с аудиоанализом с датасетом с аудиофичами

In [23]:
analysis_df = pd.read_pickle("../../data/big_genres_audio_analysis_df.pickle")
features_df = pd.read_pickle("../../data/big_genres_tracks_df.pickle")

In [34]:
merged_df = pd.merge(features_df, analysis_df, on="track_id")
merged_df = merged_df.drop(["time_signature_y"], axis=1).rename(
    {"time_signature_x": "time_signature"}, axis=1
)
merged_df.to_pickle("../../data/big_genres_merged_df.pickle")